# Math 579 Advanced Mathematical Analysis – Project
---
**Date:** 2025-04-21
**Name:** Samisoni Palu  
**Instructor:** Dr. Sun


In [5]:
import chess
import json
import numpy as np
import torch.nn as nn

## Utils
Board data
---
`generate_uci_move_list`
- list all possible moves as a 2-tuple of from_square, to_square
- tweaked for promotion move counts
- will Include Redundant or Invalid Moves
    - **`a1a1`, `d4d4`**, etc. → no actual move is made
    - **Illegal under any real game condition**
    - Also includes nonsense like `h2h8` (rook-style moves for pawns)

But remember:  
> You’re not saying “these are all valid moves”  
> **this is the full move *vocabulary***—the possible *labels* in a classification task.
> That is, our **vocabulary** list is all the two-tuple pairings of squares, e.g. (`h2g3`), along with other special moves

Our total vocabulary size count includes the sum of
- 64x64 (each pairing of squares)
- 8 (number of columns) x 2 (white-black promotions) x 4 (choices of upgrade) x 3 (capture types) - 16 (edge cases)

We should expect our logits vector to have size **4272**. 

**Benefits of the chosen Vocabulary**

1. Simplicity in Output Shape
    - 1-to-1 mapping: index ↔ UCI
    - You don’t need dynamic output heads or custom decoders
    - You can store logits as `torch.tensor([4672])` and just mask out illegal ones at runtime

2. Consistency
    - Your label space is fixed across:
      - Training
      - Inference
      - Evaluation

3. Non-moves Never Get Trained On
    - No master ever plays `a1a1`
    - So those output indices **never get gradient updates**
    - They just sit in the model—harmless dead neurons

**Why You Might Remove Redundant Moves**

1. Smaller Output Space
    - Saves compute on final linear layer and softmax
    - Slightly faster training (maybe)

2. Model Capacity Allocation
    - You force the network to **only ever consider valid move templates**
    - Could lead to sharper learning curve

But you pay with **more complexity**:
- Dynamic move indexing
- Pre-mask needs to align with training mask
- Harder debugging


In [6]:
def generate_uci_move_list():
    all_moves = set()
    for from_sq in chess.SQUARES:
        for to_sq in chess.SQUARES:
            move = chess.Move(from_sq, to_sq)
            all_moves.add(move.uci())
            # Add promotions
            for promo in [chess.QUEEN, chess.ROOK, chess.BISHOP, chess.KNIGHT]:
                from_rank = chess.square_rank(from_sq)
                to_rank = chess.square_rank(to_sq)
                from_file = chess.square_file(from_sq)
                to_file = chess.square_file(to_sq)
                # Only allow forward promotion (white or black)
                if (from_rank , to_rank) in [(6, 7), (1, 0)]:  # white/black promotion ranks
                    if abs(from_file - to_file) <= 1:         # straight or diagonal
                        promo_move = chess.Move(from_sq, to_sq, promotion=promo)
                        all_moves.add(promo_move.uci())
    return sorted(all_moves)


def save_move_index_map(path="data/move_index_map.json"):
    moves = generate_uci_move_list()
    uci_to_index = {uci: i for i, uci in enumerate(moves)}
    with open(path, "w") as f:
        json.dump(uci_to_index, f)

In [7]:
move_list = generate_uci_move_list()
print('Logits size = ',len(move_list))

Logits size =  4272


In [8]:
PIECE_TO_IDX = {
    None: 0,
    chess.PAWN: 1,
    chess.KNIGHT: 2,
    chess.BISHOP: 3,
    chess.ROOK: 4,
    chess.QUEEN: 5,
    chess.KING: 6,
}

def encode_board(board: chess.Board):
    board_array = np.zeros((8, 8), dtype=np.int64)
    
    for square in chess.SQUARES:
        piece = board.piece_at(square)
        row = 7 - (square // 8)
        col = square % 8

        if piece is not None:
            base = PIECE_TO_IDX[piece.piece_type]
            offset = 0 if piece.color == chess.WHITE else 6
            board_array[row][col] = base + offset
        else:
            board_array[row][col] = 0  # empty

    return board_array  # shape: [8,8] of ints in [0,12]


In [9]:
board = chess.Board()
encode_board(board)

array([[10,  8,  9, 11, 12,  9,  8, 10],
       [ 7,  7,  7,  7,  7,  7,  7,  7],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 1,  1,  1,  1,  1,  1,  1,  1],
       [ 4,  2,  3,  5,  6,  3,  2,  4]])

In [10]:
class PolicyNet(nn.Module):
    def __init__(self, embedding_dim=32, num_moves=4672):
        super().__init__()
        self.embed = nn.Embedding(13, embedding_dim)  # 13 tokens -> vector
        self.fc = nn.Sequential(
            nn.Flatten(),                # [8,8,32] -> [2048]
            nn.Linear(8*8*embedding_dim, 512),
            nn.ReLU(),
            nn.Linear(512, num_moves)   # Final logits
        )

    def forward(self, x):  # x: [B, 8, 8]
        x = self.embed(x)  # [B, 8, 8, D]
        return self.fc(x)


We want a neural network that takes:
- Input: an `8×8` grid of piece tokens (entries are integers from 0 to 12)
- Output: a **4672-dimensional logits vector**

---

### CLASS STRUCTURE: `PolicyNet`

```python
class PolicyNet(nn.Module):
    def __init__(self, embedding_dim=32, num_moves=4272):
```

- **`embedding_dim=32`**: each board square will be represented by a **32-dimensional vector**.
- **`num_moves=4272`**: the size of **output layer**, corresponding to all possible moves.

---

```python
        super().__init__()
```

- Standard for initializing the parent class (`nn.Module`).

---

### Embedding Layer

```python
        self.embed = nn.Embedding(13, embedding_dim)
```

- This layer turns each square’s integer (0–12) into a vector of dimension `[embedding_dim]`.
- `[8,8]` board → `[8,8,32]` tensor.

---

### Fully Connected Network (MLP)
These are our hidden and output logits layer. 
```python
        self.fc = nn.Sequential(
            nn.Flatten(),                
            nn.Linear(8*8*embedding_dim, 512),
            nn.ReLU(),
            nn.Linear(512, num_moves)
        )
```

#### `nn.Flatten()`
- Converts `[B, 8, 8, 32]` into `[B, 2048]`, similar to .view
- Needed to feed into `Linear` layers

#### `nn.Linear(2048, 512)`
- Fully connected layer reducing 2048 features to 512 neurons

#### `nn.ReLU()`
- Non-linear activation to let the model learn more complex patterns

#### `nn.Linear(512, num_moves)`
- Final layer: predicts logit value for each of the 4672 moves

---

### `forward` Function

```python
    def forward(self, x):  # x: [B, 8, 8]
        x = self.embed(x)  # [B, 8, 8, D]
        x = x.permute(0, 3, 1, 2)  # Optional: [B, D, 8, 8]
        return self.fc(x)
```

- **Input**: `x` is a batch of boards, shape `[batch_size, 8, 8]`
- **Embedding**: turns each square into a vector: `[B, 8, 8, 32]`
- **FC Network**: outputs a `[B, 4272]` tensor of logits

---

## Summary

- network **understands piece identity** through embeddings.
- it **flattens** the board to make a prediction using fully connected layers.
- The output is a **score for every possible move**, and later **mask illegal ones**.

## DATA SETS

We build X and Y like so: 

X is a board state, the model gets the output of the given input (logits) and compares with given Y (from data set). If the given string contains a full game, then we can structure our input, output pairing as pairings of adjacent moves (in correct order). For instance, a given game string may contain a list of board states:

\begin{equation} \text{Given:}\ S_1,\ S_2,...\ ,S_n \end{equation}

Then, we can order our X and Y vector as the following: 

\begin{align*}
X &= [S_1,...,S_n]\\
Y &= [S_2,...,S_{n-1}]
\end{align*}


In [28]:
with open(r"C:\Users\samip\Documents\quick-maffs\neural_nets\makemorechessmoves\data\raw_games\Carlsen.pgn", "r") as file:
    content = chess.pgn.read_game(file)[0]
    board = content.board()
    for move in content.mainline_moves(): 
            #print(move)
            board_tensor = encode_board(board)
            print(board_tensor)


[[10  8  9 11 12  9  8 10]
 [ 7  7  7  7  7  7  7  7]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  1  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 1  1  1  0  1  1  1  1]
 [ 4  2  3  5  6  3  2  4]]
[[10  8  9 11 12  9  8 10]
 [ 7  7  7  7  7  7  7  7]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  1  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 1  1  1  0  1  1  1  1]
 [ 4  2  3  5  6  3  2  4]]
[[10  8  9 11 12  9  8 10]
 [ 7  7  7  7  7  7  7  7]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  1  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 1  1  1  0  1  1  1  1]
 [ 4  2  3  5  6  3  2  4]]
[[10  8  9 11 12  9  8 10]
 [ 7  7  7  7  7  7  7  7]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  1  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 1  1  1  0  1  1  1  1]
 [ 4  2  3  5  6  3  2  4]]
[[10  8  9 11 12  9  8 10]
 [ 7  7  7  7  7  7  7  7]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  1  0  0  0  